FFT c2c

In [1]:
import numpy as np
import argparse

def main(N=1000, save_bins=True):
    rng = np.random.default_rng()
    A = 1.0 + rng.standard_normal((N, N), dtype=np.float64)
    C = np.fft.fft2(A)

    print(f"N = {N}")
    print(f"C[0,0] (DC) = {C[0,0].real:.6e} + {C[0,0].imag:.6e}i")
    print(f"C[0,1]      = {C[0,1].real:.6e} + {C[0,1].imag:.6e}i")

    if save_bins:
        np.save("A.npy", A)
        np.save("C.npy", C)
        A.tofile("A.bin")
        C.tofile("C.bin")
        print("Saved A.npy, C.npy, A.bin, C.bin")

if __name__ == "__main__":
    # Ignore unknown args (e.g., Jupyter’s)
    parser = argparse.ArgumentParser(add_help=False)
    parser.add_argument("-N", "--size", type=int, default=1000)
    parser.add_argument("--no-save", action="store_true")
    args, _unknown = parser.parse_known_args()

    main(N=args.size, save_bins=not args.no_save)


N = 1000
C[0,0] (DC) = 9.989248e+05 + 0.000000e+00i
C[0,1]      = 1.270386e+03 + -4.082025e+02i
Saved A.npy, C.npy, A.bin, C.bin


Reconstruction of matrix using FFT c2c

In [3]:

def compute_metrics(A, C):
    # NumPy's ifft2 includes the 1/(N*N) normalization, so ifft2(fft2(A)) ≈ A
    A_rec = np.fft.ifft2(C)
    # discard tiny numerical imaginary residue
    A_rec = A_rec.real

    err = A_rec - A

    # Absolute errors
    rmse_abs = np.sqrt(np.mean(err**2))
    rmedse_abs = np.sqrt(np.median(err**2))

    # Relative errors (protect against very small denominators)
    denom = np.maximum(np.abs(A), 1e-12)
    rel = err / denom
    rmse_rel = np.sqrt(np.mean(rel**2))
    rmedse_rel = np.sqrt(np.median(rel**2))

    # For sanity: how big was the imaginary residue before .real?
    max_imag = np.abs(np.fft.ifft2(C).imag).max()

    return rmse_abs, rmedse_abs, rmse_rel, rmedse_rel, max_imag

def main(N=1000, seed=None, save=False, load=False):
    rng = np.random.default_rng(seed)

    if load:
        A = np.load("A.npy")
        C = np.load("C.npy")
    else:
        # Build A ~ N(1,1)
        A = 1.0 + rng.standard_normal((N, N), dtype=np.float64)
        # Complex-to-complex 2D FFT (unnormalized forward)
        C = np.fft.fft2(A)
        if save:
            np.save("A.npy", A)
            np.save("C.npy", C)

    rmse_abs, rmedse_abs, rmse_rel, rmedse_rel, max_imag = compute_metrics(A, C)

    # Print a couple of reference values
    print(f"N = {A.shape[0]}")
    print(f"C[0,0] (DC) ≈ sum(A) = {C[0,0].real:.6e} + {C[0,0].imag:.6e}i")

    # Requested metrics
    print("\nErrors (square root of mean/median squared errors):")
    print(f"RMSE (absolute):                 {rmse_abs:.6e}")
    print(f"√(median squared absolute err):  {rmedse_abs:.6e}")
    print(f"RMSE (relative):                 {rmse_rel:.6e}")
    print(f"√(median squared relative err):  {rmedse_rel:.6e}")
    print(f"\nMax imaginary residue in ifft2:  {max_imag:.6e}")

if __name__ == "__main__":
    # Be robust to extra args (e.g., Jupyter kernels) by ignoring unknowns
    parser = argparse.ArgumentParser(add_help=True)
    parser.add_argument("-N", "--size", type=int, default=1000, help="Matrix size N (creates N×N)")
    parser.add_argument("--seed", type=int, default=None, help="RNG seed for reproducibility")
    parser.add_argument("--save", action="store_true", help="Save A.npy and C.npy")
    parser.add_argument("--load", action="store_true", help="Load A.npy and C.npy instead of generating new A")
    args, _unknown = parser.parse_known_args()

    main(N=args.size, seed=args.seed, save=args.save, load=args.load)


N = 1000
C[0,0] (DC) ≈ sum(A) = 9.995481e+05 + 0.000000e+00i

Errors (square root of mean/median squared errors):
RMSE (absolute):                 5.369850e-16
√(median squared absolute err):  4.440892e-16
RMSE (relative):                 2.896986e-13
√(median squared relative err):  3.582086e-16

Max imaginary residue in ifft2:  1.837814e-15


Reconstruction of matrix using r2c FFT

In [2]:
import numpy as np
import argparse
def main(N=1000, seed=42, save=True):
    rng = np.random.default_rng(seed)
    # Generate A ~ N(1,1)
    A = 1.0 + rng.standard_normal((N, N), dtype=np.float64)

    # Perform 2-D real-to-complex FFT
    R = np.fft.rfft2(A)   # shape: (N, N//2 + 1), dtype=complex128

    print(f"N = {N}")
    print(f"Shape of R (rfft2 result): {R.shape}")
    print(f"R[0,0] (DC)     = {R[0,0].real:.6e} + {R[0,0].imag:.6e}i")
    print(f"R[0,1]          = {R[0,1].real:.6e} + {R[0,1].imag:.6e}i")

    if save:
        np.save("A.npy", A)
        np.save("R.npy", R)
        A.tofile("A.bin")
        R.astype(np.complex128).tofile("R.bin")
        print("Saved A.npy, R.npy, A.bin, R.bin")

if __name__ == "__main__":
    main()


N = 1000
Shape of R (rfft2 result): (1000, 501)
R[0,0] (DC)     = 1.000098e+06 + 0.000000e+00i
R[0,1]          = -1.081230e+03 + 9.735750e+02i
Saved A.npy, R.npy, A.bin, R.bin


Reconstruction of matrix by inverse c2r FFT 

In [1]:
import argparse, os
import numpy as np

def rms_and_rmed(abs_err, rel_err):
    rmse_abs  = np.sqrt(np.mean(abs_err**2))
    rmedse_abs = np.sqrt(np.median(abs_err**2))
    rmse_rel  = np.sqrt(np.mean(rel_err**2))
    rmedse_rel = np.sqrt(np.median(rel_err**2))
    return rmse_abs, rmedse_abs, rmse_rel, rmedse_rel

def main(N=1000, seed=42, load=True):
    rng = np.random.default_rng(seed)

    if load and os.path.exists("A.npy") and os.path.exists("R.npy"):
        A = np.load("A.npy")
        R = np.load("R.npy")
    else:
        A = 1.0 + rng.standard_normal((N, N), dtype=np.float64)
        R = np.fft.rfft2(A)
        np.save("A.npy", A); np.save("R.npy", R)

    # Inverse real 2D FFT (r2c -> real)
    A_rec = np.fft.irfft2(R, s=(A.shape[0], A.shape[1]))

    err = A_rec - A
    denom = np.maximum(np.abs(A), 1e-12)  # protect against tiny values
    rel  = err / denom

    rmse_abs, rmedse_abs, rmse_rel, rmedse_rel = rms_and_rmed(np.abs(err), np.abs(rel))

    print(f"N = {A.shape[0]}")
    print("Errors (square root of mean/median squared errors):")
    print(f"RMSE (absolute):                 {rmse_abs:.6e}")
    print(f"√(median squared absolute err):  {rmedse_abs:.6e}")
    print(f"RMSE (relative):                 {rmse_rel:.6e}")
    print(f"√(median squared relative err):  {rmedse_rel:.6e}")
    print(f"Reconstruction ok (allclose)?    {np.allclose(A, A_rec, rtol=1e-12, atol=1e-15)}")

if __name__ == "__main__":
    p = argparse.ArgumentParser()
    p.add_argument("-N", "--size", type=int, default=1000)
    p.add_argument("--seed", type=int, default=42)
    p.add_argument("--no-load", dest="load", action="store_false")
    args, _ = p.parse_known_args()
    main(N=args.size, seed=args.seed, load=args.load)


N = 1000
Errors (square root of mean/median squared errors):
RMSE (absolute):                 6.368609e-16
√(median squared absolute err):  4.440892e-16
RMSE (relative):                 3.154822e-13
√(median squared relative err):  4.131280e-16
Reconstruction ok (allclose)?    False


Are you reaching machine precision in point 2 and 4? If not, try to comment on why?

Yes (for absolute error).
Your inverse FFT reconstructions showed absolute RMSE ≈ 5×10⁻¹⁶ and median ≈ 4×10⁻¹⁶, which is on the order of double precision machine epsilon (≈ 2.22×10⁻¹⁶). That’s as good as it gets, up to small accumulation/round-off from the FFT algorithm.
The relative RMSE (~10⁻¹³) looks larger only because you divide by some small |Aᵢⱼ|—relative error magnifies when denominators are small. Numerically, the reconstruction is at machine precision.

What is the value of C[0,0] or R[0,0]? Meaning?

C[0.0] or R[0.0] is the DC (zero frequency) component of the Fourier Transform and is equal to sum of all elements of TO. For N=1000, it is typically around 10E6, which corresponds to the total average level of  input data. 

Reduce A to 6×6 and get C from R

In [1]:
import numpy as np

def c_from_r(R):
    """
    Rebuild full complex FFT C (N x N) from R = rfft2(A) for even N.
    R shape: (N, N//2 + 1). Assumes NumPy's axis order (kx along rows, ky along cols).
    """
    N, W = R.shape  # W = N//2 + 1
    assert N % 2 == 0 and W == N//2 + 1, "This helper assumes even N."

    C = np.empty((N, N), dtype=np.complex128)

    # Left half (ky = 0..N//2) is stored directly in R
    C[:, :W] = R

    # Prepare the kx-negation index: (-kx) mod N
    # This is [0, N-1, N-2, ..., 1] (note: not a plain reverse)
    idx_neg = (-np.arange(N)) % N

    # Right half (ky = N//2+1 .. N-1) via Hermitian symmetry
    # ky_sym = N - ky_right, and use kx -> (-kx) mod N
    for ky_right in range(W, N):
        ky_sym = N - ky_right
        C[:, ky_right] = np.conj(C[idx_neg, ky_sym])

    return C

# --- quick test ---
if __name__ == "__main__":
    N = 6
    rng = np.random.default_rng(0)
    A = 1.0 + rng.standard_normal((N, N))
    R = np.fft.rfft2(A)
    C_true = np.fft.fft2(A)
    C_rec  = c_from_r(R)
    print("allclose:", np.allclose(C_rec, C_true, rtol=1e-12, atol=1e-12))


allclose: True
